Reference
- https://www.lucypark.kr/slides/2014-pyconkr/
- https://github.com/danrasband/krtpy
- https://github.com/sublee/hangulize
- https://github.com/what-studio/tossi
- https://github.com/bluedisk/hangul-toolkit
- https://github.com/zhangkaiyulw/kroman-py
- https://github.com/iki/unidecode
- http://findyou.readthedocs.io/ko/latest/dparser.html

# Make sure you use Python 3!

In [1]:
import sys
print(sys.version)

3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 13:14:59) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]


# Hangulization
- 영어를 제외한 외국어 => 한글

- 영어 => 한글 라이브러리 없음
    - https://github.com/danrasband/krtpy (Python3 x)
    - https://github.com/muik/transliteration (TensorFlow Seq2Seq... too heavy)

In [2]:
from hangulize import hangulize

In [3]:
hangulize(u"Giro d'Italia", 'ita')

'지로 디탈리아'

In [4]:
hangulize(u'オオサカ', 'jpn')

'오사카'

In [5]:
hangulize(u'przyjaciół', 'pol')

'프시야치우'

# Romanization
- 한국어 => 영어
- 외래어의 보다 자연스러운 처리 가능
- 영어를 기반으로 하는 라이브러리를 쓰고자 할 때 사용 가능

In [6]:
import kroman

In [7]:
kroman.parse('손목시계')

'son-mog-si-gye'

In [8]:
kroman.parse('바나나')

'ba-na-na'

In [9]:
kroman.parse('안녕 헨리?')

'an-nyeong hen-ri?'

In [10]:
# 임의의 단어를 영어로
from unidecode import unidecode

In [11]:
unidecode('패스트캠퍼스')

'paeseuteukaempeoseu'

In [12]:
unidecode('아이스크림 먹고 싶다')

'aiseukeurim meoggo sipda'

# 조사 선택
- 명사/동사만으로 문장을 보다 매끄럽게 하고자 할 때

In [13]:
import tossi

In [14]:
tossi.pick('김치', '은')

'는'

In [15]:
tossi.postfix('집', '으로')

'집으로'

In [16]:
eun = tossi.parse('은')
eun

<Particle: 은(는)>

In [17]:
eun['김치']

'는'

ex) 주어 + 목적어 + 좋아한다

In [18]:
words1 = ['나', '사탕', '먹고있다']
words2 = ['저', '복숭아', '좋아해요']

In [19]:
def combiner(words):
    w1, w2, w3 = words
    return " ".join([tossi.postfix(w1, '은'), tossi.postfix(w2, '을'), w3])

In [20]:
combiner(words1)

'나는 사탕을 먹고있다'

In [21]:
combiner(words2)

'저는 복숭아를 좋아해요'

# hangul-toolkit

In [22]:
import hgtk

## Compose/Decompose
- 초성, 중성, 종성 합성 및 분리

In [23]:
hgtk.letter.decompose("감")

('ㄱ', 'ㅏ', 'ㅁ')

In [24]:
hgtk.letter.compose(*hgtk.letter.decompose("감"))

'감'

In [25]:
hgtk.text.decompose("학교종이 땡땡땡! hello world 1234567890 ㅋㅋ!") # ᴥ: whitespace

'ㅎㅏㄱᴥㄱㅛᴥㅈㅗㅇᴥㅇㅣᴥ ㄸㅐㅇᴥㄸㅐㅇᴥㄸㅐㅇᴥ! hello world 1234567890 ㅋᴥㅋᴥ!'

In [26]:
hgtk.text.compose('ㅎㅏㄱ 교 종이 땡 ㄸㅐㅇ 때ㅇ 땡')

'학 교 종이 땡 땡 때ㅇ 땡'

## Checker
- 한글인지/한자인지/외래어인지/영어인지/받침이 있는지 체크

In [27]:
hgtk.checker.is_hangul('한글입니다')

True

In [28]:
hgtk.checker.is_hangul('no한글입니다')

False

In [29]:
hgtk.checker.has_batchim('한')

True

In [30]:
hgtk.checker.has_batchim('하')

False

In [31]:
hgtk.checker.is_latin1('abcdefghijklmnopqrstuvwxyz')

True

In [32]:
messi = 'Lionel Messi'
if hgtk.checker.is_latin1(messi):
    print(hangulize(messi, 'spa'))

리오넬 메시


#### by Lucy Park (https://www.lucypark.kr/slides/2014-pyconkr/#32)

한자, 한글, 영문이 섞여 있는 경우 Multilingual NLP에서 제안하는 다양한 접근법이 있습니다.

간단한 꼼수 한 가지는: "한자 전처리, 영문 후처리"

- 한자는 transliterate하고 (ex: '丁新闻' -> 정신문)
- 한국어 처리를 한 후에 (ex: POS tagging)
- 영문으로 분류된 tag에 한해 따로 stemming 등을 거침

In [42]:
import nltk
stemmer = nltk.stem.SnowballStemmer('english')

In [35]:
from konlpy.tag import Twitter
twitter = Twitter()

In [65]:
sentence = 'Recurrent Neural Network 가 뭐하는 건가요?'

In [70]:
def en_kr_tokenizer(sentence):
    out_words = []
    words = twitter.pos(sentence)
    for word, pos in words:
        if hgtk.checker.is_latin1(word) or pos == 'Alpha':
            word = stemmer.stem(word)
        out_words.append(word)
    return out_words

In [71]:
en_kr_tokenizer(sentence)

['recurr', 'neural', 'network', '가', '뭐', '하는', '건가', '요', '?']

# Parser (구문분석)
- [dparser](http://findyou.readthedocs.io/ko/latest/dparser.html)
- [SyntaxNet](https://github.com/tensorflow/models/tree/master/syntaxnet) + [Sejong Corpus](https://ithub.korean.go.kr) + a

In [1]:
import requests
import json

In [2]:
def parse(sentence):
    dparser_url = 'http://parser.datanada.com/parse?version=1&string='
    response = requests.get(dparser_url+sentence)
    return json.loads(response.text)

In [3]:
parse("안녕하세요 반갑습니다.")

[{'deprel': 'VP', 'form': '안녕', 'head': '4', 'id': '1', 'pos': 'NNG'},
 {'deprel': 'VP', 'form': '하', 'head': '1', 'id': '2', 'pos': 'XSA'},
 {'deprel': 'VP', 'form': '세요', 'head': '2', 'id': '3', 'pos': 'EP+EC'},
 {'deprel': 'ROOT', 'form': '반갑', 'head': '0', 'id': '4', 'pos': 'VA'},
 {'deprel': 'VP', 'form': '습니다', 'head': '4', 'id': '5', 'pos': 'EF'},
 {'deprel': 'VP', 'form': '.', 'head': '5', 'id': '6', 'pos': 'SF'}]

# etc.

- Curated list of Korean NLP https://github.com/datanada/Awesome-Korean-NLP
- 국민대 강승식 교수님 형태소 분석기 http://nlp.kookmin.ac.kr/HAM/kor/
- 울산대 형태소 분석기 http://nlplab.ulsan.ac.kr/doku.php?id=utagger
- Rouzeta https://shleekr.github.io/